In [51]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from itertools import cycle
import plotly.graph_objects as go
import os
import sys

plt.rcParams.update({"figure.dpi": 150})
idx = pd.IndexSlice
pd.options.display.max_columns = None

# module_path = os.path.abspath(os.path.join('../util'))
# # if module_path not in sys.path:
# sys.path.append(module_path)
# module_path

# LOAD DATA


In [52]:
from util.load_scenarios import load_scenario_df
from util.load_evaluations import load_benchmark_df

eval_df = load_benchmark_df("../data/evaluation/benchmarking/default")
eval_df = eval_df.add_prefix("eval.")

scenario_df = load_scenario_df("../data/definition/routes_devtest_sliced.xml")
scenario_df = scenario_df.add_prefix("def.")

df = eval_df.join(scenario_df, how='inner', validate="1:m")
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 17920 entries, (10, 'False', '0', 0) to (20, 'True', '9', 131)
Data columns (total 29 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   eval.route_id                           17920 non-null  object 
 1   eval.status                             17920 non-null  object 
 2   eval.infractions.collisions_layout      17920 non-null  object 
 3   eval.infractions.collisions_pedestrian  17920 non-null  object 
 4   eval.infractions.collisions_vehicle     17920 non-null  object 
 5   eval.infractions.outside_route_lanes    17920 non-null  object 
 6   eval.infractions.red_light              17920 non-null  object 
 7   eval.infractions.route_dev              17920 non-null  object 
 8   eval.infractions.route_timeout          17920 non-null  object 
 9   eval.infractions.stop_infraction        17920 non-null  object 
 10  eval.infractions.vehi

# Show flakiness


In [53]:
hf = df.xs((20, "True"))

agg = hf.select_dtypes('number').groupby('route_index')
std_of_scenarios = agg['eval.driving_score'].std()
# agg['eval.driving_score'].mean()
most_flaky_scearios = std_of_scenarios.sort_values().tail(n=10)
print(most_flaky_scearios)
flay_scenario_ids = most_flaky_scearios.index.to_list()

route_index
95    0.172017
81    0.193218
90    0.206559
39    0.206559
6     0.235976
73    0.244156
65    0.280190
88    0.291638
8     0.326745
61    0.328476
Name: eval.driving_score, dtype: float64


In [54]:
current_scenario = flay_scenario_ids[4]
print(f'{current_scenario=}')
most_flaky = hf.xs(current_scenario, level='route_index')


def drop_boring_columns(df):
    # if a column has one elements its boring lets drop it
    for series_name, series in df.items():
        if len(series.apply(str).unique()) == 1:
            df = df.drop(series_name, axis=1)
            # print(f"Dropped: {series_name}")

    return df


most_flaky = drop_boring_columns(most_flaky)

# crashes = most_flaky['eval.infractions.collisions_vehicle'].apply(str).unique()
# print(crashes)
most_flaky

current_scenario=6


,eval.infractions.collisions_vehicle,eval.infractions.red_light,eval.duration_game,eval.duration_system,eval.score_composed,eval.score_penalty,eval.driving_score,eval.driving_score_error
rep,,,,,,,,
0,[],[],155.050002,536.154901,100.0,1.00,1.00,0.218
1,[Agent collided against object with type=vehic...,[],155.050002,546.453636,60.0,0.60,0.60,0.182
2,[],[],155.100002,538.723685,100.0,1.00,1.00,0.218
3,[Agent collided against object with type=vehic...,[],155.000002,545.373864,60.0,0.60,0.60,0.182
4,[Agent collided against object with type=vehic...,"[Agent ran a red light 1473 at (x=-0.82, y=202...",126.400002,444.104573,42.0,0.42,0.42,0.362
5,[],[],154.900002,524.478811,100.0,1.00,1.00,0.218
6,[],[],155.450002,557.934449,100.0,1.00,1.00,0.218
7,[Agent collided against object with type=vehic...,[],155.100002,535.100192,60.0,0.60,0.60,0.182
8,[Agent collided against object with type=vehic...,[],155.400002,527.815523,60.0,0.60,0.60,0.182
